In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
# --- 1. Cargar los datos ---
ratings = pd.read_csv('casa_cultura_data/ratings.csv')
copies = pd.read_csv('casa_cultura_data/copies(ejemplares).csv')         # copy_id, book_id
books = pd.read_csv('casa_cultura_data/books.csv')           # book_id, title
# Eliminar nulos
ratings.dropna(subset=['user_id', 'copy_id', 'rating'], inplace=True)
copies.dropna(subset=['copy_id', 'book_id'], inplace=True)
books.dropna(subset=['book_id', 'title'], inplace=True)
# Eliminar ratings fuera del rango
ratings = ratings[(ratings['rating'] >= 1) & (ratings['rating'] <= 5)]
# Eliminar duplicados si los hubiera
ratings.drop_duplicates(inplace=True)
copies.drop_duplicates(inplace=True)
books.drop_duplicates(subset='book_id', inplace=True)


# --- 2. Unir ratings con books ---
# Paso 1: Obtener book_id a partir de copy_id
ratings_books = ratings.merge(copies, on='copy_id', how='left')

# Paso 2: Unir con libros para tener títulos
ratings_books = ratings_books.merge(books[['book_id', 'title']], on='book_id', how='left')

# --- 3. Filtrar solo valoraciones positivas (ej: >= 4) ---
ratings_books = ratings_books[ratings_books['rating'] >= 4]

# --- 4. Crear tabla de transacciones usuario-libros ---
basket = ratings_books.groupby(['user_id', 'title'])['rating'].count().unstack().fillna(0)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

# --- 5. Aplicar Apriori ---
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# --- 6. Generar reglas ---
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
rules = rules.sort_values(by='lift', ascending=False)

# --- 7. Mostrar recomendaciones ---
def recomendar_libros(basado_en_libro, top_n=5):
    basado_en_libro = basado_en_libro.lower()
    recomendados = []

    for _, row in rules.iterrows():
        if basado_en_libro in [x.lower() for x in row['antecedents']]:
            recomendados.extend(row['consequents'])

    # Eliminar duplicados y limitar a N
    recomendados = list(set(recomendados))
    return recomendados[:top_n]

# Ejemplo de uso:
print("Si te gustó 'The Hobbit', también podrías leer:")
print(recomendar_libros('The Hobbit'))



In [ ]:
# --- 2. Unir ratings con books ---
# Paso 1: Obtener book_id a partir de copy_id
ratings_books = ratings.merge(copies, on='copy_id', how='left')

# Paso 2: Unir con libros para tener títulos
ratings_books = ratings_books.merge(books[['book_id', 'title']], on='book_id', how='left')




In [ ]:
# --- 3. Filtrar solo valoraciones positivas (ej: >= 4) ---
ratings_books = ratings_books[ratings_books['rating'] >= 4]


In [ ]:
# --- 4. Crear tabla de transacciones usuario-libros ---
basket = ratings_books.groupby(['user_id', 'title'])['rating'].count().unstack().fillna(0)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

In [ ]:
# --- 5. Aplicar Apriori ---
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

In [ ]:
# --- 6. Generar reglas ---
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
rules = rules.sort_values(by='lift', ascending=False)

In [ ]:
# --- 7. Mostrar recomendaciones ---
def recomendar_libros(basado_en_libro, top_n=5):
    basado_en_libro = basado_en_libro.lower()
    recomendados = []

    for _, row in rules.iterrows():
        if basado_en_libro in [x.lower() for x in row['antecedents']]:
            recomendados.extend(row['consequents'])

    # Eliminar duplicados y limitar a N
    recomendados = list(set(recomendados))
    return recomendados[:top_n]

In [ ]:
# Ejemplo de uso:
print("Si te gustó 'The Hobbit', también podrías leer:")
print(recomendar_libros('The Hobbit'))